# Simulation of the Henry Saltwater Intrusion Problem

This problem simulates the classic Henry problem (Henry, 1964) for variable-density groundwater flow and solute transport. The MODFLOW 6 simulations are based on the hydraulic-head formulation for variable-density flow as presented by Langevin and others (2020).

Henry, H., 1964, Effects of dispersion on salt encroachment in coastal aquifers: Sea Water in Coastal Aquifers, U.S. Geol. Surv. Supply Pap., 1613-C, C71-C84.

Langevin, C.D., Panday, S., and Provost, A.M., 2020, Hydraulic-head formulation for density-dependent flow and transport: Groundwater, v. 58, no. 3, p. 349–362, https://doi.org/10.1111/gwat.12967.

## Imports

In [ ]:
# imports
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation
import numpy as np
import flopy
from flopy.plot.styles import styles
%matplotlib inline

## Specify Parameters

In [ ]:
# Model units
length_units = "centimeters"
time_units = "days"

# Model parameters
nper = 1  # Number of periods
nstp = 500  # Number of time steps
perlen = 0.5  # Simulation time length ($d$)
nlay = 40  # Number of layers
nrow = 1  # Number of rows
ncol = 80  # Number of columns
system_length = 2.0  # Length of system ($m$)
system_height = 1.0
delr = system_length / ncol  # Column width ($m$)
delc = 1.0  # Row width ($m$)
delv = system_height / nlay  # Layer thickness
top = 1.0  # Top of the model ($m$)
hydraulic_conductivity = 864.0  # Hydraulic conductivity ($m d^{-1}$)
initial_concentration = 35.0  # Initial concentration (unitless)
porosity = 0.35  # porosity (unitless)
diffusion_coefficient = 0.57024  # diffusion coefficient ($m^2/d$)
inflow = 5.7024 # inflow rate m^3/d

botm = [top - k * delv for k in range(1, nlay + 1)]

nouter, ninner = 100, 300
hclose, rclose, relax = 1e-10, 1e-6, 0.97

## Construct the Simulation

In [ ]:
sim_ws = "./temp/henry"
sim_name = "henry"
sim = flopy.mf6.MFSimulation(sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")
tdis_ds = ((perlen, nstp, 1.0),)
tdis = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)

## Create the GWF Model

In [ ]:
gwf = flopy.mf6.ModflowGwf(sim, modelname="flow", save_flows=True)
ims_gwf = flopy.mf6.ModflowIms(
    sim,
    print_option="ALL",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="NONE",
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=rclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    filename=f"{gwf.name}.ims",
)
sim.register_ims_package(ims_gwf, [gwf.name])
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=0,
    k=hydraulic_conductivity,
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=initial_concentration)
pd = [(0, 0.7, 0.0, "trans", "concentration")]
buy = flopy.mf6.ModflowGwfbuy(gwf, packagedata=pd)
ghbcond = hydraulic_conductivity * delv * delc / (0.5 * delr)
ghbspd = [[(k, 0, ncol - 1), top, ghbcond, 35.0] for k in range(nlay)]
ghb = flopy.mf6.ModflowGwfghb(
    gwf,
    stress_period_data=ghbspd,
    pname="GHB-1",
    auxiliary="CONCENTRATION",
)

welspd = [[(k, 0, 0), inflow / nlay, 0.0] for k in range(nlay)]
wel = flopy.mf6.ModflowGwfwel(
    gwf,
    stress_period_data=welspd,
    pname="WEL-1",
    auxiliary="CONCENTRATION",
)
head_filerecord = f"{gwf.name}.hds"
budget_filerecord = f"{gwf.name}.bud"
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=head_filerecord,
    budget_filerecord=budget_filerecord,
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

## Create the GWT Model

In [ ]:
gwt = flopy.mf6.ModflowGwt(sim, modelname="trans")
imsgwt = flopy.mf6.ModflowIms(
    sim,
    print_option="ALL",
    outer_dvclose=hclose,
    outer_maximum=nouter,
    under_relaxation="NONE",
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=rclose,
    linear_acceleration="BICGSTAB",
    scaling_method="NONE",
    reordering_method="NONE",
    relaxation_factor=relax,
    filename=f"{gwt.name}.ims",
)
sim.register_ims_package(imsgwt, [gwt.name])
dis = flopy.mf6.ModflowGwtdis(
    gwt,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
mst = flopy.mf6.ModflowGwtmst(gwt, porosity=porosity)
ic = flopy.mf6.ModflowGwtic(gwt, strt=initial_concentration)
adv = flopy.mf6.ModflowGwtadv(gwt, scheme="UPSTREAM")
dsp = flopy.mf6.ModflowGwtdsp(gwt, xt3d_off=True, diffc=diffusion_coefficient)
sourcerecarray = [
    ("GHB-1", "AUX", "CONCENTRATION"),
    ("WEL-1", "AUX", "CONCENTRATION"),
]
ssm = flopy.mf6.ModflowGwtssm(gwt, sources=sourcerecarray)
oc = flopy.mf6.ModflowGwtoc(
    gwt,
    budget_filerecord=f"{gwt.name}.cbc",
    concentration_filerecord=f"{gwt.name}.ucn",
    concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
    saverecord=[("CONCENTRATION", "ALL")],
    printrecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")],
)

## Create the GWF-GWT Exchange

In [ ]:

gwfgwt = flopy.mf6.ModflowGwfgwt(
    sim, exgtype="GWF6-GWT6", exgmnamea=gwf.name, exgmnameb=gwt.name
)

## Write the Model Datasets

In [ ]:
sim.write_simulation()

## Run the MODFLOW 6 Simulation

In [ ]:
sim.run_simulation()

## Load and Plot the Results

In [ ]:
# load output
qx, qy, qz = None, None, None
head_all = gwf.output.head().get_alldata()
bud = gwf.output.budget()
times = bud.times
spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)
conc_all = gwt.output.concentration().get_alldata()

In [ ]:
figure_size = (6, 4)
with styles.USGSMap():

    fig = plt.figure(figsize=figure_size)
    fig.tight_layout()

    conc = conc_all[-1]

    ax = fig.add_subplot(1, 1, 1, aspect="equal")
    pxs = flopy.plot.PlotCrossSection(model=gwf, ax=ax, line={"row": 0})
    pxs.plot_array(conc, cmap="jet")
    levels = [35 * f for f in [0.01, 0.1, 0.5, 0.9, 0.99]]
    cs = pxs.contour_array(
        conc, levels=levels, colors="w", linewidths=1.0, linestyles="-"
    )
    ax.set_xlabel("x position (m)")
    ax.set_ylabel("z position (m)")
    plt.clabel(cs, fmt="%4.2f", fontsize=6)

## Create Animation

In [ ]:
def get_title(time_in_days):
    time_in_minutes = time_in_days * 24. * 60.
    return f"Time = {time_in_minutes:0.2f} mins"

times_to_show = times[::10]
ntimes = len(times_to_show)
bud = gwf.output.budget()

nplotrows = 1
nplotcols = 1
fig, axes = plt.subplots(
    nrows=nplotrows, 
    ncols=nplotcols, 
    figsize=figure_size, 
    layout="constrained"
)
ax = axes
ax.set_ylabel(r'z')
title = ax.set_title(get_title(times[0]))
ax.set_xlabel(r'x')
ax.set_xlim(0, 2.0)
ax.set_ylim(0, 1.0)
ax.set_aspect(1.0)

# # plot persistent items
plot_array_dict = {
    "cmap": "jet",
    "masked_values": [1.e30],
}
colorbar_text_size = 10
pmv = flopy.plot.PlotCrossSection(gwt, line={"row": 0}, ax=ax)
pmv.plot_inactive()
pa = pmv.plot_array(
    conc_all[0], 
    vmin=0., 
    vmax=35., 
    **plot_array_dict
)
cb = plt.colorbar(pa, shrink=0.5)
cb.ax.set_ylabel('salinity (g/L)', rotation=270, fontsize=colorbar_text_size, labelpad=15)
cb.ax.tick_params(labelsize=colorbar_text_size)

def animate(i):
    itime = times.index(times_to_show[i])
    title = ax.set_title(get_title(times[itime]))
    pa = pmv.plot_array(
        conc_all[itime], 
        vmin=0., 
        vmax=35., 
        **plot_array_dict
    )
    spdis = bud.get_data(text='DATA-SPDIS')[itime]
    qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)
    pmv.plot_vector(qx, qy, qz, normalize=False, color="white")

    return

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=ntimes)
plt.close()

# Create and show the animation in the notebook
HTML(ani.to_jshtml())

In [ ]:
def run_model(diffc):
    gwt.dsp.diffc = diffc
    gwt.dsp.write()
    sim.run_simulation(silent=True)
    df = gwf.output.list().get_dataframes()[0]
    total_out = df.TOTAL_OUT.iloc[0]
    return total_out
run_model(.5)

In [ ]:
diff_cs = np.arange(.1, 1, .1)
total_outs = []
for diffc in diff_cs:
    total_out = run_model(diffc)
    total_outs.append(total_out)

In [ ]:
plt.plot(diff_cs, total_outs, 'bo-')